# Open-FF Company Index

In [ ]:
from math import log10, floor
import core.get_google_map as ggmap

def round_sig(x, sig=2):
    try:
        if abs(x)>=1:
            out =  int(round(x, sig-int(floor(log10(abs(x))))-1))
            return f"{out:,d}" # does the right thing with commas
        else: # fractional numbers
            return str(round(x, sig-int(floor(log10(abs(x))))-1))
    except:
        return x
    
# used to insert links of google maps into tables
def make_clickable(val):
    try:
        if val[:4]=='http':
            return '<a href="{}" target="_blank">{}</a>'.format(val,'map')
    except:
        return val
    return val

def getLink(row):
    return ggmap.getSearchLink(row.bgLatitude,row.bgLongitude)

def getCatLink(cas):
#    s = f'https://qbobioyuz1dh57rst8exeg-on.drv.tw/open_FF_catalog/{cas}/analysis_{cas}.html'
#    s = f'file://C:/MyDocs/sandbox/data/O&G/open-FF/tmp/website/{cas}/analysis_{cas}.html'
    s = f'{cas}/analysis_{cas}.html'
    return ggmap.wrap_URL_in_html(s,'Analysis')

def getDataLink(cas):
    s = f'{cas}/data.csv'
    return ggmap.wrap_URL_in_html(s,'csv file')

def getChemIDLink(cas):
    s = f'https://chem.nlm.nih.gov/chemidplus/rn/{cas}'
    return ggmap.wrap_URL_in_html(s,'ChemID')

def getMapLink(row):
    s = getLink(row)
    return ggmap.wrap_URL_in_html(s,'map')


def xlate_to_str(inp,sep=' ',trunc=False,tlen=20):
    try:
        l = list(inp)
        out = ''
        for s in l:
            if trunc:
                if len(s)>tlen:
                    s = s[:tlen-3]+ '...'
            out+= s+sep
    except:
        return ''
    return out[:-(len(sep))]

In [ ]:
import pandas as pd
import numpy as np
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from itables import show as iShow
import itables.options as opt


import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 500)

import core.Analysis_set as ana_set
df_cas = ana_set.Catalog_set().get_set()


|Explanation of columns in the index|
| :---: |

| Column      | Description |
| :----: | :-------- |
|*raw*| company name as it appears in the raw FracFocus (fields: OperatorName or Supplier)|
|*translated*|company name as translated by Open-FF (fields: bgOperatorName, bgSupplier, primarySupplier)|
|*number of OperatorName records*| how many times this raw value appears as the Operator value|
|*number of Supplier records*| how many times this raw value appears as the Supplier value|

In [ ]:
# fn = './sources/company_xlate.csv'
# xlate = pd.read_csv(fn,quotechar='$')
# xlate = xlate.drop('status',axis=1)
# xlate.columns = ['raw','translated']
gbOp = df_cas.groupby(['OperatorName','bgOperatorName'],as_index=False)['bgCAS'].count()
gbOp.columns = ['raw','translated','counts in OperatorName']
gbSup = df_cas.groupby(['Supplier','bgSupplier'],as_index=False)['bgCAS'].count()
gbSup.columns = ['raw','translated','counts in Supplier']
xlate =pd.merge(gbOp,gbSup, on=['translated','raw'],how='outer').reset_index(drop=True)
xlate = xlate.fillna(0)
iShow(xlate,maxBytes=0)